<a href="https://colab.research.google.com/github/yunmokoo/SPY-Forecast_MARS_LSTM/blob/main/implementing_pandas_SPY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests > make_outputrerquest.txt 2>&1
!pip install yfinance > make_outputyfinance.txt 2>&1
!pip install transformers > make_outputtransformers.txt 2>&1
!pip install xformers > make_outputxformers.txt 2>&1


In [ ]:
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz > wget_output.txt 2>&1
!tar -xzvf ta-lib-0.4.0-src.tar.gz > tar_output.txt 2>&1
%cd ta-lib
!./configure --prefix=/usr > config_output.txt 2>&1
!make > make_outputtalib.txt 2>&1
!make install > makeInstall_output.txt 2>&1
!pip install TA-Lib > pipInstall_output.txt 2>&1

/content/ta-lib


In [ ]:
import requests
import json
import yfinance as yf
#import talib

import tensorflow as tf
import numpy as np
import pandas as pd
import torch
import matplotlib as plt

from transformers import *
from transformers import GenerationMixin
from transformers import TFGenerationMixin
from transformers import FlaxGenerationMixin

from sklearn.preprocessing import MinMaxScaler

From TA-lib...

1. MA                  All Moving Average
2. BBANDS              Bollinger Bands
3. MACD                Moving Average Convergence/Divergence
4. SMA                 Simple Moving Average
5. EMA                 Exponential Moving Average

In [ ]:

data_SPY = yf.download("SPY", start = "2001-01-01", end = "2023-01-01")


print(data_SPY)

[*********************100%***********************]  1 of 1 completed
                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2001-01-02  132.000000  132.156250  127.562500  128.812500   84.662994   
2001-01-03  128.312500  136.000000  127.656250  135.000000   88.729759   
2001-01-04  134.937500  135.468750  133.000000  133.546875   87.774658   
2001-01-05  133.468750  133.625000  129.187500  129.187500   84.909447   
2001-01-08  129.875000  130.187500  127.687500  130.187500   85.566689   
...                ...         ...         ...         ...         ...   
2022-12-23  379.649994  383.059998  378.029999  382.910004  380.042480   
2022-12-27  382.790009  383.149994  379.649994  381.399994  378.543793   
2022-12-28  381.329987  383.390015  376.420013  376.660004  373.839264   
2022-12-29  379.630005  384.350006  379.079987  383.440002  380.568512   
2022-12-30  380.640015  382.579987  378.429

# Implementing Pandas.

In [ ]:
df = pd.DataFrame(data_SPY)
df

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2001-01-02,132.000000,132.156250,127.562500,128.812500,84.662956,8737500
2001-01-03,128.312500,136.000000,127.656250,135.000000,88.729729,19431600
2001-01-04,134.937500,135.468750,133.000000,133.546875,87.774651,9219000
2001-01-05,133.468750,133.625000,129.187500,129.187500,84.909454,12911400
2001-01-08,129.875000,130.187500,127.687500,130.187500,85.566727,6625300
...,...,...,...,...,...,...
2022-12-23,379.649994,383.059998,378.029999,382.910004,380.042480,59857300
2022-12-27,382.790009,383.149994,379.649994,381.399994,378.543793,51638200
2022-12-28,381.329987,383.390015,376.420013,376.660004,373.839294,70911500


In [ ]:
df.shape

(5535, 6)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5535 entries, 2001-01-02 to 2022-12-30
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       5535 non-null   float64
 1   High       5535 non-null   float64
 2   Low        5535 non-null   float64
 3   Close      5535 non-null   float64
 4   Adj Close  5535 non-null   float64
 5   Volume     5535 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 302.7 KB


In [ ]:
#if columns are cut off..

pd.set_option('display.max_columns', 6)
df

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2001-01-02,132.000000,132.156250,127.562500,128.812500,84.662971,8737500
2001-01-03,128.312500,136.000000,127.656250,135.000000,88.729744,19431600
2001-01-04,134.937500,135.468750,133.000000,133.546875,87.774704,9219000
2001-01-05,133.468750,133.625000,129.187500,129.187500,84.909470,12911400
2001-01-08,129.875000,130.187500,127.687500,130.187500,85.566696,6625300
...,...,...,...,...,...,...
2022-12-23,379.649994,383.059998,378.029999,382.910004,380.042480,59857300
2022-12-27,382.790009,383.149994,379.649994,381.399994,378.543793,51638200
2022-12-28,381.329987,383.390015,376.420013,376.660004,373.839264,70911500


In [ ]:
df.head()   #this prints first 5 rows

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2001-01-02,132.00000,132.15625,127.56250,128.812500,84.662971,8737500
2001-01-03,128.31250,136.00000,127.65625,135.000000,88.729744,19431600
2001-01-04,134.93750,135.46875,133.00000,133.546875,87.774704,9219000
2001-01-05,133.46875,133.62500,129.18750,129.187500,84.909470,12911400
2001-01-08,129.87500,130.18750,127.68750,130.187500,85.566696,6625300


In [ ]:
df.head(10) #prints first 10 rows

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2001-01-02,132.000000,132.156250,127.562500,128.812500,84.662971,8737500
2001-01-03,128.312500,136.000000,127.656250,135.000000,88.729744,19431600
2001-01-04,134.937500,135.468750,133.000000,133.546875,87.774704,9219000
2001-01-05,133.468750,133.625000,129.187500,129.187500,84.909470,12911400
2001-01-08,129.875000,130.187500,127.687500,130.187500,85.566696,6625300
2001-01-09,131.046875,131.500000,129.421875,129.843750,85.340752,5702400
2001-01-10,129.000000,132.125000,128.812500,132.125000,86.840141,8746100
2001-01-11,131.093750,133.484375,131.093750,132.250000,86.922325,7245100
2001-01-12,132.687500,133.718750,131.281250,132.000000,86.757980,7244000


In [ ]:
df.tail()   #last 5 rows

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-12-23,379.649994,383.059998,378.029999,382.910004,380.042480,59857300
2022-12-27,382.790009,383.149994,379.649994,381.399994,378.543793,51638200
2022-12-28,381.329987,383.390015,376.420013,376.660004,373.839264,70911500
2022-12-29,379.630005,384.350006,379.079987,383.440002,380.568512,66970900
2022-12-30,380.640015,382.579987,378.429993,382.429993,379.566040,84022200


In [ ]:
df['Close']   #or df.Close

Date
2001-01-02    128.812500
2001-01-03    135.000000
2001-01-04    133.546875
2001-01-05    129.187500
2001-01-08    130.187500
                 ...    
2022-12-23    382.910004
2022-12-27    381.399994
2022-12-28    376.660004
2022-12-29    383.440002
2022-12-30    382.429993
Name: Close, Length: 5535, dtype: float64

In [ ]:
type(df['Close'])

pandas.core.series.Series

In [ ]:
df[['Close', 'Open']]

,Close,Open
Date,,
2001-01-02,128.812500,132.000000
2001-01-03,135.000000,128.312500
2001-01-04,133.546875,134.937500
2001-01-05,129.187500,133.468750
2001-01-08,130.187500,129.875000
...,...,...
2022-12-23,382.910004,379.649994
2022-12-27,381.399994,382.790009
2022-12-28,376.660004,381.329987


In [ ]:
df.columns

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

In [ ]:
df.iloc[0]

Open         1.320000e+02
High         1.321562e+02
Low          1.275625e+02
Close        1.288125e+02
Adj Close    8.466297e+01
Volume       8.737500e+06
Name: 2001-01-02 00:00:00, dtype: float64

In [ ]:
df.iloc[[0, 1]]

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2001-01-02,132.0000,132.15625,127.56250,128.8125,84.662971,8737500
2001-01-03,128.3125,136.00000,127.65625,135.0000,88.729744,19431600


In [ ]:
df.iloc[[0, 1], 2]  #This printed Low

Date
2001-01-02    127.56250
2001-01-03    127.65625
Name: Low, dtype: float64

iloc searches by integer location,

with loc, we search by label.

In [ ]:
df.loc['2001-01-02']

Open         1.320000e+02
High         1.321562e+02
Low          1.275625e+02
Close        1.288125e+02
Adj Close    8.466297e+01
Volume       8.737500e+06
Name: 2001-01-02 00:00:00, dtype: float64

In [ ]:
df.loc[['2001-01-02', '2001-01-03']]

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2001-01-02,132.0000,132.15625,127.56250,128.8125,84.662971,8737500
2001-01-03,128.3125,136.00000,127.65625,135.0000,88.729744,19431600


In [ ]:
df.loc[['2001-01-02', '2001-01-03'], ['Close', 'Open']]   #['Close', 'Open'] order is respected

,Close,Open
Date,,
2001-01-02,128.8125,132.0000
2001-01-03,135.0000,128.3125


In [ ]:
df['Close']

Date
2001-01-02    128.812500
2001-01-03    135.000000
2001-01-04    133.546875
2001-01-05    129.187500
2001-01-08    130.187500
                 ...    
2022-12-23    382.910004
2022-12-27    381.399994
2022-12-28    376.660004
2022-12-29    383.440002
2022-12-30    382.429993
Name: Close, Length: 5535, dtype: float64

In [ ]:
df['Close'].value_counts()

131.970001    6
114.199997    5
82.750000     4
126.660004    4
112.580002    4
             ..
109.430000    1
110.989998    1
109.940002    1
111.300003    1
382.429993    1
Name: Close, Length: 4762, dtype: int64

In [ ]:
df.loc['2001-01-02', 'Close']

128.8125

In [ ]:
df.loc[['2001-01-02','2001-01-03','2001-01-04','2001-01-05'], 'Close']

Date
2001-01-02    128.812500
2001-01-03    135.000000
2001-01-04    133.546875
2001-01-05    129.187500
Name: Close, dtype: float64

In [ ]:
df.loc['2001-01-02':'2001-01-12', 'Close':'Volume']    #Utilizing slicing

,Close,Adj Close,Volume
Date,,,
2001-01-02,128.812500,84.662971,8737500
2001-01-03,135.000000,88.729744,19431600
2001-01-04,133.546875,87.774704,9219000
2001-01-05,129.187500,84.909470,12911400
2001-01-08,130.187500,85.566696,6625300
2001-01-09,129.843750,85.340752,5702400
2001-01-10,132.125000,86.840141,8746100
2001-01-11,132.250000,86.922325,7245100
2001-01-12,132.000000,86.757980,7244000


In [ ]:
df.set_index('Close')

,Open,High,Low,Adj Close,Volume
Close,,,,,
128.812500,132.000000,132.156250,127.562500,84.662971,8737500
135.000000,128.312500,136.000000,127.656250,88.729744,19431600
133.546875,134.937500,135.468750,133.000000,87.774704,9219000
129.187500,133.468750,133.625000,129.187500,84.909470,12911400
130.187500,129.875000,130.187500,127.687500,85.566696,6625300
...,...,...,...,...,...
382.910004,379.649994,383.059998,378.029999,380.042480,59857300
381.399994,382.790009,383.149994,379.649994,378.543793,51638200
376.660004,381.329987,383.390015,376.420013,373.839264,70911500


In [ ]:
df.index

DatetimeIndex(['2001-01-02', '2001-01-03', '2001-01-04', '2001-01-05',
               '2001-01-08', '2001-01-09', '2001-01-10', '2001-01-11',
               '2001-01-12', '2001-01-16',
               ...
               '2022-12-16', '2022-12-19', '2022-12-20', '2022-12-21',
               '2022-12-22', '2022-12-23', '2022-12-27', '2022-12-28',
               '2022-12-29', '2022-12-30'],
              dtype='datetime64[ns]', name='Date', length=5535, freq=None)

In [ ]:
df.set_index('Close', inplace=True)   #Sets index into 'Close'

In [ ]:
df.index

Float64Index([          128.8125,              135.0,         133.546875,
                        129.1875,           130.1875,          129.84375,
                         132.125,             132.25,              132.0,
                       132.84375,
              ...
               383.2699890136719,  380.0199890136719,  380.5400085449219,
               386.2300109863281,  380.7200012207031,  382.9100036621094,
               381.3999938964844,  376.6600036621094, 383.44000244140625,
              382.42999267578125],
             dtype='float64', name='Close', length=5535)

In [ ]:
df.loc[132.0, 'Volume']


7244000.0

In [ ]:
df.reset_index(inplace=True)    #Resets the index, if run multiple times it assins new numerical index to it
df                              # a.k.a. duplicates. You can use 'if' to check beforehand.

ValueError: ignored

In [ ]:
if df.index.name is not None:
  df.reset_index(inplace=True)

In [ ]:
df['Close'] >= 150    #Can filter out or detect the point of change.

Date
2001-01-02    False
2001-01-03    False
2001-01-04    False
2001-01-05    False
2001-01-08    False
              ...  
2022-12-23     True
2022-12-27     True
2022-12-28     True
2022-12-29     True
2022-12-30     True
Name: Close, Length: 5535, dtype: bool

In [ ]:
filt = (df['Close'] >= 150)
df[filt]

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2007-05-03,149.970001,150.399994,149.729996,150.350006,109.577545,86569700
2007-05-04,150.750000,151.119995,150.220001,150.919998,109.992973,96409000
2007-05-07,150.880005,151.199997,150.809998,150.949997,110.014870,63461400
2007-05-08,150.580002,150.919998,150.130005,150.750000,109.869026,80584000
2007-05-09,150.639999,152.820007,150.369995,151.160004,110.167938,102070100
...,...,...,...,...,...,...
2022-12-23,379.649994,383.059998,378.029999,382.910004,380.042480,59857300
2022-12-27,382.790009,383.149994,379.649994,381.399994,378.543793,51638200
2022-12-28,381.329987,383.390015,376.420013,376.660004,373.839264,70911500


In [ ]:
df.loc[filt, 'Close']

Date
2007-05-03    150.350006
2007-05-04    150.919998
2007-05-07    150.949997
2007-05-08    150.750000
2007-05-09    151.160004
                 ...    
2022-12-23    382.910004
2022-12-27    381.399994
2022-12-28    376.660004
2022-12-29    383.440002
2022-12-30    382.429993
Name: Close, Length: 2590, dtype: float64

In [ ]:
filt = (df['Close'] >= 150) & (df['Close'] <= 250)
df.loc[filt, 'Close']

Date
2007-05-03    150.350006
2007-05-04    150.919998
2007-05-07    150.949997
2007-05-08    150.750000
2007-05-09    151.160004
                 ...    
2020-03-23    222.949997
2020-03-24    243.149994
2020-03-25    246.789993
2020-04-01    246.149994
2020-04-03    248.190002
Name: Close, Length: 1279, dtype: float64

In [ ]:
filt = (df['Close'] >= 150) | (df['Close'] <= 250)
df.loc[filt, 'Close']

Date
2001-01-02    128.812500
2001-01-03    135.000000
2001-01-04    133.546875
2001-01-05    129.187500
2001-01-08    130.187500
                 ...    
2022-12-23    382.910004
2022-12-27    381.399994
2022-12-28    376.660004
2022-12-29    383.440002
2022-12-30    382.429993
Name: Close, Length: 5535, dtype: float64

In [ ]:
filt = (df['Close'] >= 150) & (df['Close'] <= 250)
df.loc[~filt, 'Close']

Date
2001-01-02    128.812500
2001-01-03    135.000000
2001-01-04    133.546875
2001-01-05    129.187500
2001-01-08    130.187500
                 ...    
2022-12-23    382.910004
2022-12-27    381.399994
2022-12-28    376.660004
2022-12-29    383.440002
2022-12-30    382.429993
Name: Close, Length: 4256, dtype: float64

since ~ was added before 'filt', the difference in the result could be seen through the length. One with the ~ before 'filt' has 4256 length.

In [ ]:
df.columns

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

## Uppercasing column labels

In [ ]:
df.columns  = [x.upper() for x in df.columns]
df

,OPEN,HIGH,LOW,CLOSE,ADJ CLOSE,VOLUME
Date,,,,,,
2001-01-02,132.000000,132.156250,127.562500,128.812500,84.663002,8737500
2001-01-03,128.312500,136.000000,127.656250,135.000000,88.729774,19431600
2001-01-04,134.937500,135.468750,133.000000,133.546875,87.774712,9219000
2001-01-05,133.468750,133.625000,129.187500,129.187500,84.909462,12911400
2001-01-08,129.875000,130.187500,127.687500,130.187500,85.566704,6625300
...,...,...,...,...,...,...
2022-12-23,379.649994,383.059998,378.029999,382.910004,380.042480,59857300
2022-12-27,382.790009,383.149994,379.649994,381.399994,378.543793,51638200
2022-12-28,381.329987,383.390015,376.420013,376.660004,373.839264,70911500


In [ ]:
df.columns = df.columns.str.replace(' ', '_')
df

,OPEN,HIGH,LOW,CLOSE,ADJ_CLOSE,VOLUME
Date,,,,,,
2001-01-02,132.000000,132.156250,127.562500,128.812500,84.662979,8737500
2001-01-03,128.312500,136.000000,127.656250,135.000000,88.729767,19431600
2001-01-04,134.937500,135.468750,133.000000,133.546875,87.774681,9219000
2001-01-05,133.468750,133.625000,129.187500,129.187500,84.909447,12911400
2001-01-08,129.875000,130.187500,127.687500,130.187500,85.566711,6625300
...,...,...,...,...,...,...
2022-12-23,379.649994,383.059998,378.029999,382.910004,380.042480,59857300
2022-12-27,382.790009,383.149994,379.649994,381.399994,378.543793,51638200
2022-12-28,381.329987,383.390015,376.420013,376.660004,373.839264,70911500


In [ ]:
df.rename(columns={'OPEN': 'OPEN_PRICE', 'CLOSE': 'CLOSE_PRICE'}, inplace=True)

,OPEN_PRICE,HIGH,LOW,CLOSE_PRICE,ADJ_CLOSE,VOLUME
Date,,,,,,
2001-01-02,132.000000,132.156250,127.562500,128.812500,84.662979,8737500
2001-01-03,128.312500,136.000000,127.656250,135.000000,88.729767,19431600
2001-01-04,134.937500,135.468750,133.000000,133.546875,87.774681,9219000
2001-01-05,133.468750,133.625000,129.187500,129.187500,84.909447,12911400
2001-01-08,129.875000,130.187500,127.687500,130.187500,85.566711,6625300
...,...,...,...,...,...,...
2022-12-23,379.649994,383.059998,378.029999,382.910004,380.042480,59857300
2022-12-27,382.790009,383.149994,379.649994,381.399994,378.543793,51638200
2022-12-28,381.329987,383.390015,376.420013,376.660004,373.839264,70911500




---

now, changing the row elements.

---



In [ ]:
df.loc['2001-01-02'] = [123, 123, 123, 123, 123, 123]
df

,OPEN,HIGH,LOW,CLOSE,ADJ_CLOSE,VOLUME
Date,,,,,,
2001-01-02,123.000000,123.000000,123.000000,123.000000,123.000000,123
2001-01-03,128.312500,136.000000,127.656250,135.000000,88.729767,19431600
2001-01-04,134.937500,135.468750,133.000000,133.546875,87.774681,9219000
2001-01-05,133.468750,133.625000,129.187500,129.187500,84.909447,12911400
2001-01-08,129.875000,130.187500,127.687500,130.187500,85.566711,6625300
...,...,...,...,...,...,...
2022-12-23,379.649994,383.059998,378.029999,382.910004,380.042480,59857300
2022-12-27,382.790009,383.149994,379.649994,381.399994,378.543793,51638200
2022-12-28,381.329987,383.390015,376.420013,376.660004,373.839264,70911500


In [ ]:
df.loc['2001-01-02', ['OPEN', 'CLOSE']] = [999, 999]
df

,OPEN,HIGH,LOW,CLOSE,ADJ_CLOSE,VOLUME
Date,,,,,,
2001-01-02,999.000000,123.000000,123.000000,999.000000,123.000000,123
2001-01-03,128.312500,136.000000,127.656250,135.000000,88.729767,19431600
2001-01-04,134.937500,135.468750,133.000000,133.546875,87.774681,9219000
2001-01-05,133.468750,133.625000,129.187500,129.187500,84.909447,12911400
2001-01-08,129.875000,130.187500,127.687500,130.187500,85.566711,6625300
...,...,...,...,...,...,...
2022-12-23,379.649994,383.059998,378.029999,382.910004,380.042480,59857300
2022-12-27,382.790009,383.149994,379.649994,381.399994,378.543793,51638200
2022-12-28,381.329987,383.390015,376.420013,376.660004,373.839264,70911500


In [ ]:
df.loc['2001-01-02', 'VOLUME'] = 12345
df

,OPEN,HIGH,LOW,CLOSE,ADJ_CLOSE,VOLUME
Date,,,,,,
2001-01-02,999.000000,123.000000,123.000000,999.000000,123.000000,12345
2001-01-03,128.312500,136.000000,127.656250,135.000000,88.729767,19431600
2001-01-04,134.937500,135.468750,133.000000,133.546875,87.774681,9219000
2001-01-05,133.468750,133.625000,129.187500,129.187500,84.909447,12911400
2001-01-08,129.875000,130.187500,127.687500,130.187500,85.566711,6625300
...,...,...,...,...,...,...
2022-12-23,379.649994,383.059998,378.029999,382.910004,380.042480,59857300
2022-12-27,382.790009,383.149994,379.649994,381.399994,378.543793,51638200
2022-12-28,381.329987,383.390015,376.420013,376.660004,373.839264,70911500


In [ ]:
filt = (df['CLOSE'] >= 150)
df[filt]['VOLUME']

Date
2001-01-02       12345
2007-05-03    86569700
2007-05-04    96409000
2007-05-07    63461400
2007-05-08    80584000
                ...   
2022-12-23    59857300
2022-12-27    51638200
2022-12-28    70911500
2022-12-29    66970900
2022-12-30    84022200
Name: VOLUME, Length: 2591, dtype: int64

In [ ]:
df.loc[filt, 'VOLUME'] = 950
df

,OPEN,HIGH,LOW,CLOSE,ADJ_CLOSE,VOLUME
Date,,,,,,
2001-01-02,999.000000,123.000000,123.000000,999.000000,123.000000,950
2001-01-03,128.312500,136.000000,127.656250,135.000000,88.729767,19431600
2001-01-04,134.937500,135.468750,133.000000,133.546875,87.774681,9219000
2001-01-05,133.468750,133.625000,129.187500,129.187500,84.909447,12911400
2001-01-08,129.875000,130.187500,127.687500,130.187500,85.566711,6625300
...,...,...,...,...,...,...
2022-12-23,379.649994,383.059998,378.029999,382.910004,380.042480,950
2022-12-27,382.790009,383.149994,379.649994,381.399994,378.543793,950
2022-12-28,381.329987,383.390015,376.420013,376.660004,373.839264,950


In [ ]:
df['VOLUME'] = df['VOLUME'].astype(str)
df['LENGTH'] = df['VOLUME'].str.len()

In [ ]:
df

,OPEN,HIGH,LOW,CLOSE,ADJ CLOSE,VOLUME,LENGTH
Date,,,,,,,
2001-01-02,132.000000,132.156250,127.562500,128.812500,84.663010,8737500,7
2001-01-03,128.312500,136.000000,127.656250,135.000000,88.729759,19431600,8
2001-01-04,134.937500,135.468750,133.000000,133.546875,87.774681,9219000,7
2001-01-05,133.468750,133.625000,129.187500,129.187500,84.909462,12911400,8
2001-01-08,129.875000,130.187500,127.687500,130.187500,85.566681,6625300,7
...,...,...,...,...,...,...,...
2022-12-23,379.649994,383.059998,378.029999,382.910004,380.042480,59857300,8
2022-12-27,382.790009,383.149994,379.649994,381.399994,378.543793,51638200,8
2022-12-28,381.329987,383.390015,376.420013,376.660004,373.839294,70911500,8


In [ ]:
def update_VOLUME(VOLUME):
  return VOLUME.upper()

In [ ]:
df['VOLUME'].apply(update_VOLUME)

Date
2001-01-02     8737500
2001-01-03    19431600
2001-01-04     9219000
2001-01-05    12911400
2001-01-08     6625300
                ...   
2022-12-23    59857300
2022-12-27    51638200
2022-12-28    70911500
2022-12-29    66970900
2022-12-30    84022200
Name: VOLUME, Length: 5535, dtype: object

In [ ]:
df['VOLUME'] = df['VOLUME'].apply(lambda x: x.lower())

In [ ]:
df.apply(len)

OPEN         5535
HIGH         5535
LOW          5535
CLOSE        5535
ADJ CLOSE    5535
VOLUME       5535
dtype: int64

In [ ]:
len(df['VOLUME'])

5535

In [ ]:
df.apply(len, axis='columns')

Date
2001-01-02    7
2001-01-03    7
2001-01-04    7
2001-01-05    7
2001-01-08    7
             ..
2022-12-23    7
2022-12-27    7
2022-12-28    7
2022-12-29    7
2022-12-30    7
Length: 5535, dtype: int64

In [ ]:
df.apply(pd.Series.min)

OPEN         67.949997
HIGH              70.0
LOW          67.099998
CLOSE        68.110001
ADJ CLOSE    51.568043
VOLUME       100028200
LENGTH               7
dtype: object

In [ ]:
df_reset = df.reset_index()


In [ ]:
df

,OPEN,HIGH,LOW,CLOSE,ADJ CLOSE,VOLUME,LENGTH
Date,,,,,,,
2001-01-02,132.000000,132.156250,127.562500,128.812500,84.663010,8737500,7
2001-01-03,128.312500,136.000000,127.656250,135.000000,88.729759,19431600,8
2001-01-04,134.937500,135.468750,133.000000,133.546875,87.774681,9219000,7
2001-01-05,133.468750,133.625000,129.187500,129.187500,84.909462,12911400,8
2001-01-08,129.875000,130.187500,127.687500,130.187500,85.566681,6625300,7
...,...,...,...,...,...,...,...
2022-12-23,379.649994,383.059998,378.029999,382.910004,380.042480,59857300,8
2022-12-27,382.790009,383.149994,379.649994,381.399994,378.543793,51638200,8
2022-12-28,381.329987,383.390015,376.420013,376.660004,373.839294,70911500,8


In [ ]:
df.apply(pd.Series.min)

OPEN         67.949997
HIGH              70.0
LOW          67.099998
CLOSE        68.110001
ADJ CLOSE    51.568043
VOLUME       100028200
LENGTH               7
dtype: object

In [ ]:
df['VOLUME'] = pd.to_numeric(df['VOLUME'], errors='coerce')

In [ ]:
df.apply(pd.Series.min)

OPEN         6.795000e+01
HIGH         7.000000e+01
LOW          6.710000e+01
CLOSE        6.811000e+01
ADJ CLOSE    5.156803e+01
VOLUME       3.303100e+06
dtype: float64

In [ ]:
df.apply(lambda x: x.min())

OPEN         6.795000e+01
HIGH         7.000000e+01
LOW          6.710000e+01
CLOSE        6.811000e+01
ADJ CLOSE    5.156803e+01
VOLUME       3.303100e+06
dtype: float64

In [ ]:
df.applymap(len)

TypeError: ignored

In [ ]:
df['VOLUME'].map({8737500: 123, 19431600: 123})


Date
2001-01-02    123.0
2001-01-03    123.0
2001-01-04      NaN
2001-01-05      NaN
2001-01-08      NaN
              ...  
2022-12-23      NaN
2022-12-27      NaN
2022-12-28      NaN
2022-12-29      NaN
2022-12-30      NaN
Name: VOLUME, Length: 5535, dtype: float64

## Sorting

In [ ]:
df.sort_values(by='CLOSE')

,OPEN,HIGH,LOW,CLOSE,ADJ CLOSE,VOLUME
Date,,,,,,
2009-03-09,67.949997,70.000000,67.730003,68.110001,51.568043,379905300
2009-03-05,70.099998,71.730003,68.169998,68.800003,52.090454,485549400
2009-03-06,69.400002,70.449997,67.099998,68.919998,52.181309,490470000
2009-03-03,71.610001,71.699997,69.639999,70.070000,53.052010,443761000
2009-03-02,72.519997,73.919998,70.370003,70.599998,53.453281,426452600
...,...,...,...,...,...,...
2021-12-28,477.720001,478.809998,476.059998,476.869995,465.742950,47274600
2021-12-27,472.059998,477.309998,472.010010,477.260010,466.123840,56808600
2021-12-29,476.980011,478.559998,475.920013,477.480011,466.338684,54503000


In [ ]:
df.sort_values(by='CLOSE', ascending=False)

,OPEN,HIGH,LOW,CLOSE,ADJ CLOSE,VOLUME
Date,,,,,,
2022-01-03,476.299988,477.850006,473.850006,477.709991,466.563324,72668200
2022-01-04,479.220001,479.980011,475.579987,477.549988,466.407043,71178700
2021-12-29,476.980011,478.559998,475.920013,477.480011,466.338684,54503000
2021-12-27,472.059998,477.309998,472.010010,477.260010,466.123840,56808600
2021-12-28,477.720001,478.809998,476.059998,476.869995,465.742950,47274600
...,...,...,...,...,...,...
2009-03-02,72.519997,73.919998,70.370003,70.599998,53.453281,426452600
2009-03-03,71.610001,71.699997,69.639999,70.070000,53.052010,443761000
2009-03-06,69.400002,70.449997,67.099998,68.919998,52.181309,490470000


In [ ]:
df.sort_values(by=['CLOSE', 'OPEN'], ascending=False)

,OPEN,HIGH,LOW,CLOSE,ADJ CLOSE,VOLUME
Date,,,,,,
2022-01-03,476.299988,477.850006,473.850006,477.709991,466.563324,72668200
2022-01-04,479.220001,479.980011,475.579987,477.549988,466.407043,71178700
2021-12-29,476.980011,478.559998,475.920013,477.480011,466.338684,54503000
2021-12-27,472.059998,477.309998,472.010010,477.260010,466.123840,56808600
2021-12-28,477.720001,478.809998,476.059998,476.869995,465.742950,47274600
...,...,...,...,...,...,...
2009-03-02,72.519997,73.919998,70.370003,70.599998,53.453281,426452600
2009-03-03,71.610001,71.699997,69.639999,70.070000,53.052010,443761000
2009-03-06,69.400002,70.449997,67.099998,68.919998,52.181309,490470000
